In [1]:
import numpy as np


In [2]:
Q = np.array([[-1,1,0,0],
         [2,-3,1,0],
         [0,2,-3,1],
         [0,0,2,-2]])
I = np.identity(4)

P = (1/3) * Q + I
P

array([[0.66666667, 0.33333333, 0.        , 0.        ],
       [0.66666667, 0.        , 0.33333333, 0.        ],
       [0.        , 0.66666667, 0.        , 0.33333333],
       [0.        , 0.        , 0.66666667, 0.33333333]])

In [3]:
np.linalg.matrix_power(P,50)

array([[0.53333333, 0.26666667, 0.13333333, 0.06666667],
       [0.53333333, 0.26666667, 0.13333333, 0.06666667],
       [0.53333333, 0.26666667, 0.13333333, 0.06666667],
       [0.53333333, 0.26666667, 0.13333333, 0.06666667]])

In [1]:
import numpy as np
import pandas as pd
from math import sqrt
from collections import Counter

In [76]:
def simulador(lambda_=1, mi_=2, tempo_total_simulacao=10000, deterministico=False):
    n = tempo_simulacao = num_chegadas = tempo_ocupado_servidor = 0
    clientes_fila, chegadas, freq_clientes, tempo_clientes_fila, tempo_clientes_sistema, tempo_espera_sistema = [], [], [], [], [], []

    exponential = (lambda mi: 1 / mi) if deterministico else (
        lambda mi: np.random.exponential(scale=1 / mi))

    trace = {'Numero de clientes na fila': [],
             'Evento': [],
             'Tempo': [],
    }
    while tempo_simulacao < tempo_total_simulacao:
        tempo_chegada = np.random.exponential(scale=1 / lambda_)
        tempo_saida = exponential(mi_)
        last_event_time = tempo_simulacao

        if n == 0 or tempo_chegada < tempo_saida:
            tempo_simulacao += tempo_chegada
            num_chegadas += 1
            tempo_clientes_sistema.append(
                (tempo_simulacao - last_event_time) * n)
            
            chegadas.append(tempo_simulacao)

            if n == 0:
                freq_clientes.append(len(clientes_fila))
                
            n += 1
            if n > 1:
                clientes_fila.append(tempo_simulacao)
                tempo_ocupado_servidor += tempo_chegada
                freq_clientes.append(len(clientes_fila))

            trace['Numero de clientes na fila'].append(n)
            trace['Evento'].append('Chegada')
            trace['Tempo'].append(tempo_simulacao)

        else:
            tempo_simulacao += tempo_saida
            tempo_clientes_sistema.append(
                (tempo_simulacao - last_event_time) * n)

            if len(clientes_fila):
                tempo_clientes_fila.append(
                    tempo_simulacao - clientes_fila.pop(0))

            tempo_espera_sistema.append(tempo_simulacao - chegadas.pop(0))
            tempo_ocupado_servidor += tempo_saida
            n -= 1
            
            trace['Numero de clientes na fila'].append(n)
            trace['Evento'].append('Saida')
            trace['Tempo'].append(tempo_simulacao)


    soma_tempo_clientes_fila = sum(tempo_clientes_fila)
    soma_tempo_clientes_sistema = sum(tempo_clientes_sistema)

    Wq = soma_tempo_clientes_fila / num_chegadas
    W = soma_tempo_clientes_sistema / num_chegadas
    L = soma_tempo_clientes_sistema / tempo_simulacao
    Lq = soma_tempo_clientes_fila / tempo_simulacao
    rho = tempo_ocupado_servidor / tempo_simulacao
    pi = Counter(freq_clientes)

    for valor, freq in pi.items():
        pi[valor] = round(freq / num_chegadas, 6)

    results = {
        'Total de Clientes': num_chegadas,
        'Clientes Restantes': n,
        'Tempo Total': tempo_simulacao,
        'Tempo Espera Sistema': tempo_espera_sistema,
        'Wq': Wq,
        'W': W,
        'L': L,
        'Lq': Lq,
        'rho': rho,
        'pi': pi,
        'trace': pd.DataFrame(trace)
    }

    return results

result = simulador(lambda_=1, mi_=2, tempo_total_simulacao=100000, deterministico=False)


In [77]:
result['trace']

,Numero de clientes na fila,Evento,Tempo
0,1,Chegada,0.488316
1,2,Chegada,0.655974
2,3,Chegada,0.678672
3,4,Chegada,1.141949
4,3,Saida,1.444059
...,...,...,...
201128,1,Chegada,99995.083765
201129,0,Saida,99995.351691
201130,1,Chegada,99998.716675
201131,0,Saida,99999.321420


In [25]:
from functions import valor_analitico

In [26]:
lambda_ =1
mi_ = 2
result = simulador(lambda_=lambda_, mi_=mi_, tempo_total_simulacao=100000, deterministico=False)

# Index
indices = ['Tempo médio do cliente no sistema (L)',
           'Tempo médio de cliente na fila (Lq)',
           'Número médio de clientes no sistema (W)',
           'Número médio de clientes na fila (Wq)',
           'Probabilidade do sistema ocupado (p)']

# Valor Simulado
data_simulado = [
        result['L'],
        result['Lq'],
        result['W'],
        result['Wq'],
        result['rho']]

def custom_round(value):
    return round(value, 2)
data_simulado = [custom_round(value) for value in data_simulado]

# Valor Analitico
data_analitico = valor_analitico(lambda_,mi_)

df = pd.DataFrame({'Simulado': data_simulado, 'Analítico': data_analitico}, index=indices)
df

,Simulado,Analítico
Tempo médio do cliente no sistema (L),1.0,1.0
Tempo médio de cliente na fila (Lq),0.5,0.5
Número médio de clientes no sistema (W),1.0,1.0
Número médio de clientes na fila (Wq),0.5,0.5
Probabilidade do sistema ocupado (p),0.5,0.5


In [19]:


# Função para calcular a fração de árvores finitas usando o seu simulador
def q1numerica(lamda, mu, numSimulacoes=60, minimoParaInfinitude=300, runs=50, max_time=2000, max_events=2000):
    listaAmostras = []

    for i in range(0, numSimulacoes):
        # Utilizando o seu simulador para obter os resultados da simulação
        results = simulador(lambda_=lamda, mi_=mu, tempo_total_simulacao=max_time)
        num_chegadas = results['Total de Clientes']
        n = results['Clientes Restantes']
        clientes_fila = results['Fila Clientes'][0] if 0 in results['Fila Clientes'] else 0

        arvoresFinitas = n
        arvoresInfinitas = 0 if clientes_fila == 0 else 1

        listaAmostras.append(arvoresFinitas / (arvoresFinitas + arvoresInfinitas))

    # Tiramos a média e intervalo de confiança
    amostras = pd.Series(listaAmostras)
    media = amostras.mean()
    z = 1.96
    s = amostras.std()
    n = numSimulacoes
    intervalo = (media - z * (s / sqrt(n)), media + z * (s / sqrt(n)))

    print("\tFração de árvores finitas (resultado numérico):", media)
    print("\tIntervalo de confiança:", intervalo)


In [14]:
results = simulador(lambda_=4, mi_=2, tempo_total_simulacao=10000)


In [21]:
q1numerica(4,2)

	Fração de árvores finitas (resultado numérico): 0.9997515140758936
	Intervalo de confiança: (0.9997498419722083, 0.9997531861795789)
